In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import AdamW
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau,ModelCheckpoint, EarlyStopping

import numpy as np
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/

'Colab Notebooks'   dataset


In [ ]:
# Defining Path
DATASET_PATH = "/content/drive/MyDrive/dataset"

In [ ]:
# ================================
# 🟢 Data Preparation
# ================================

# Define image size and batch size
IMG_SIZE = (224,224) # Resize all images to 224x224 pixel values
BATCH_SIZE = 32        # Number of images per batch

In [ ]:
# Load dataset from directory
# dataset = image_dataset_from_directory(
#     DATASET_PATH,
#     labels='inferred', # Infer labels from folder names
#     label_mode="int",
#     image_size=IMAGE_SIZE,
#     batch_size=BATCH_SIZE
# )
# Data Augmentation for images

In [ ]:
# Data Augumentation for images
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    brightness_range=[0.8, 1.2],
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    validation_split=0.2  # Splitting into train and validation sets
)

In [ ]:
# # Split the dataset into training, validation, and test sets
# train_size = 0.7
# val_size = 0.2
# test_size = 0.1

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
train_dir = "/content/drive/MyDrive/dataset/train"
val_dir = "/content/drive/MyDrive/dataset/val"
test_dir = "/content/drive/MyDrive/dataset/test"

# Define image size and batch size
IMG_SIZE = (224, 224)  # Adjust as per model
BATCH_SIZE = 32

# Data augmentation for training (to improve generalization)
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation & test (no augmentation)
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load train data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Load validation data
val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

# Load test data (only for final evaluation, not for training)
test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False  # No shuffling for test set
)


Found 5778 images belonging to 5 classes.
Found 826 images belonging to 5 classes.
Found 1656 images belonging to 5 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Define dataset path
DATASET_PATH = "/content/drive/MyDrive/dataset"

# Define constants
IMG_SIZE = (224, 224)  # Adjust based on your model
BATCH_SIZE = 32
SEED = 42  # For reproducibility

# Load training and validation datasets (80% train, 20% validation)
train_dataset = image_dataset_from_directory(
    directory=f"{DATASET_PATH}/train",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    seed=SEED
)

val_dataset = image_dataset_from_directory(
    directory=f"{DATASET_PATH}/val",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    seed=SEED
)

# Load test dataset (for final evaluation)
test_dataset = image_dataset_from_directory(
    directory=f"{DATASET_PATH}/test",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode="categorical",
    shuffle=False  # No shuffling for test set
)

# Apply prefetching for performance optimization
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


In [ ]:
# # Getting the number of batches
# total_batches = tf.data.experimental.cardinality(dataset).numpy()
# train_batches = int(total_batches * train_size)
# val_batches = int(total_batches * val_size)
# print(total_batches,train_batches,val_batches)

In [ ]:
# Extract class weights to handle imbalance
labels = train_generator.classes
class_weights = compute_class_weight('balanced',classes=np.unique(labels),y=labels)
class_weights_dict = {i:class_weights[i] for i in range(len(class_weights))}

In [ ]:
import numpy as np
unique, counts = np.unique(train_generator.classes, return_counts=True)
print(dict(zip(unique, counts)))  # {0: 500, 1: 400, 2: 600}


{0: 2286, 1: 1046, 2: 1516, 3: 757, 4: 173}


In [ ]:
from os import name
# ================================
# 🟢 Model Architecture
# ================================

# Image Input
image_input = Input(shape=(224,224,3),name="Image_Input")
base_model = EfficientNetB0(include_top=False,weights='imagenet',input_tensor=image_input)
base_model.trainable = False # Freezing EfficientNet layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256,activation='relu',kernel_regularizer=l2(0.01))(x)
x = Dropout(0.3)(x)
x = Dense(128,activation='relu',kernel_regularizer=l2(0.01))(x)
x = Dropout(0.3)(x)
image_output = Dense(64,activation='relu')(x)

# Final Output Layer (5 Classes: Grade 0-4)
output = Dense(5, activation='softmax',name="Final_Output")(image_output)


In [ ]:
# Compile Model
model = Model(inputs=image_input,outputs=output)
optimizer = AdamW(learning_rate=0.001,weight_decay=0.01)
model.compile(optimizer=optimizer,loss="categorical_crossentropy",metrics=["accuracy"])

In [ ]:
# Learning Rate Scheduler
lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',factor=0.5,patience=3,verbose=1)

In [ ]:
# 🌟 Class Weights (Modify according to dataset distribution)
class_weights = {0: 1.0, 1: 2.0, 2: 2.5, 3: 3.0, 4: 3.5}  # Adjust based on dataset imbalance

# 🌟

In [ ]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Image_Input (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ Image_Input[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling_1 (Rescaling)   │ (None, 224, 224, 3)    │              0 │ normalization[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ rescaling_1[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand    

 Total params: 4,418,984 (16.86 MB)

 Trainable params: 369,413 (1.41 MB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:



# 🌟 Callbacks for Optimization
checkpoint = ModelCheckpoint("CombinedOptimizedModel.h5", monitor='val_loss', save_best_only=True, mode='min', verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)

# 🌟 Train the Model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=25,  # You can adjust this
    class_weight=class_weights,  # Handling class imbalance
    callbacks=[checkpoint, early_stopping, lr_scheduler]
)


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.2573 - loss: 5.7410
Epoch 1: val_loss improved from inf to 1.78852, saving model to CombinedOptimizedModel.h5


181/181 ━━━━━━━━━━━━━━━━━━━━ 1956s 11s/step - accuracy: 0.2573 - loss: 5.7334 - val_accuracy: 0.2567 - val_loss: 1.7885 - learning_rate: 0.0010
Epoch 2/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2565 - loss: 3.0866
Epoch 2: val_loss improved from 1.78852 to 1.62535, saving model to CombinedOptimizedModel.h5


181/181 ━━━━━━━━━━━━━━━━━━━━ 631s 3s/step - accuracy: 0.2565 - loss: 3.0864 - val_accuracy: 0.2567 - val_loss: 1.6253 - learning_rate: 0.0010
Epoch 3/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2647 - loss: 2.9684
Epoch 3: val_loss improved from 1.62535 to 1.54835, saving model to CombinedOptimizedModel.h5


181/181 ━━━━━━━━━━━━━━━━━━━━ 610s 3s/step - accuracy: 0.2647 - loss: 2.9683 - val_accuracy: 0.2567 - val_loss: 1.5483 - learning_rate: 0.0010
Epoch 4/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2606 - loss: 2.9417
Epoch 4: val_loss improved from 1.54835 to 1.52501, saving model to CombinedOptimizedModel.h5


181/181 ━━━━━━━━━━━━━━━━━━━━ 605s 3s/step - accuracy: 0.2606 - loss: 2.9416 - val_accuracy: 0.2567 - val_loss: 1.5250 - learning_rate: 0.0010
Epoch 5/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2567 - loss: 2.9053
Epoch 5: val_loss improved from 1.52501 to 1.51699, saving model to CombinedOptimizedModel.h5


181/181 ━━━━━━━━━━━━━━━━━━━━ 619s 3s/step - accuracy: 0.2567 - loss: 2.9052 - val_accuracy: 0.2567 - val_loss: 1.5170 - learning_rate: 0.0010
Epoch 6/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2642 - loss: 2.8690
Epoch 6: val_loss did not improve from 1.51699
181/181 ━━━━━━━━━━━━━━━━━━━━ 620s 3s/step - accuracy: 0.2642 - loss: 2.8690 - val_accuracy: 0.2567 - val_loss: 1.5185 - learning_rate: 0.0010
Epoch 7/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2650 - loss: 2.8260
Epoch 7: val_loss did not improve from 1.51699
181/181 ━━━━━━━━━━━━━━━━━━━━ 607s 3s/step - accuracy: 0.2650 - loss: 2.8262 - val_accuracy: 0.2567 - val_loss: 1.5201 - learning_rate: 0.0010
Epoch 8/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2631 - loss: 2.8679
Epoch 8: val_loss improved from 1.51699 to 1.51539, saving model to CombinedOptimizedModel.h5


181/181 ━━━━━━━━━━━━━━━━━━━━ 608s 3s/step - accuracy: 0.2631 - loss: 2.8679 - val_accuracy: 0.2567 - val_loss: 1.5154 - learning_rate: 0.0010
Epoch 9/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2558 - loss: 2.8968
Epoch 9: val_loss improved from 1.51539 to 1.49395, saving model to CombinedOptimizedModel.h5


181/181 ━━━━━━━━━━━━━━━━━━━━ 622s 3s/step - accuracy: 0.2558 - loss: 2.8966 - val_accuracy: 0.2567 - val_loss: 1.4939 - learning_rate: 0.0010
Epoch 10/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2547 - loss: 2.8726
Epoch 10: val_loss did not improve from 1.49395
181/181 ━━━━━━━━━━━━━━━━━━━━ 615s 3s/step - accuracy: 0.2547 - loss: 2.8726 - val_accuracy: 0.2567 - val_loss: 1.5093 - learning_rate: 0.0010
Epoch 11/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2644 - loss: 2.8360
Epoch 11: val_loss did not improve from 1.49395
181/181 ━━━━━━━━━━━━━━━━━━━━ 627s 3s/step - accuracy: 0.2643 - loss: 2.8361 - val_accuracy: 0.2567 - val_loss: 1.5066 - learning_rate: 0.0010
Epoch 12/25
181/181 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.2705 - loss: 2.8725
Epoch 12: val_loss improved from 1.49395 to 1.49344, saving model to CombinedOptimizedModel.h5


181/181 ━━━━━━━━━━━━━━━━━━━━ 612s 3s/step - accuracy: 0.2704 - loss: 2.8725 - val_accuracy: 0.2567 - val_loss: 1.4934 - learning_rate: 0.0010
Epoch 13/25
 81/181 ━━━━━━━━━━━━━━━━━━━━ 5:02 3s/step - accuracy: 0.2665 - loss: 2.8134

In [ ]:
model.evaluate(test_generator)